## Run this code FOREVER :3

In [ ]:
import paho.mqtt.client as mqttClient
import ibmiotf.device
import time
import numpy as np
import pandas as pd
from scipy.fftpack import fft
from cloudant.client import Cloudant
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn import ensemble
import mpl_toolkits
import pickle
from pushetta import Pushetta
from pynq import Overlay
from pynq import DefaultIP
import pynq.lib.dma
from pynq import Xlnk

try:
  options = {
    "org": "YOUR_ORG",
    "type": "BBM",
    "id": "001",
    "auth-method": "token",
    "auth-token": "YOUR_TOKEN",
    "clean-session": 1
  }
  clientmqtt = ibmiotf.device.Client(options)
except ibmiotf.ConnectionException  as e:
    ...

clientmqtt.connect()
serviceUsername = "YOURNUMBER-bluemix"
servicePassword = "YOUR_SERVICE_PASSWORD"
serviceURL = "https://YOURNUMBER-bluemix.cloudant.com"
clients = Cloudant(serviceUsername, servicePassword, url=serviceURL)
clients.connect();
myDatabaseDemo=clients['YOUR_DATABASE']

"""
Data per second Check the configuration of the Arduino 
and the Raspberry because the frequency programmed in 
this program is 100 data per second and every 10 minutes
we calculate the frequency for the algorithm of machine learning
"""

i=1
j=0
k=0

data_per_second=10 # Data per Second
data_frecuency=1    # Every # Minutes 
mpa=0.3              # Maximum permissible amplitude

datain = np.empty([data_per_second*60*data_frecuency+1, 3], dtype="float64") 

k=((60/data_frecuency)*24)-1

def on_connect(client, userdata, flags, rc):
 
    if rc == 0:
 
        print("Connected to broker")
 
        global Connected                #Use global variable
        Connected = True                #Signal connection 
 
    else:
 
        print("Connection failed")
 
def on_message(client, userdata, message):
    
    global data_per_second
    global data_frecuency
    global mpa
    
    """Editable Variables"""

    
    
    """Editable Variables"""
    
    data_per_day=(60/data_frecuency)*24
    datanumber=data_per_second*60*data_frecuency
    sublin=message.payload
    subline=sublin.decode()
    global i,j,k
    
    if(i <= (datanumber)):
        if (subline[0:2] == "x:" and len(subline)<8 and len(subline)>5):
            try:
                datain[i,j]=float(subline[2:len(subline)])
            except IndexError:
                #index Error, Don´t Care
                k=k
            j+=1
        elif (subline[0:2] == "y:" and len(subline)<8 and len(subline)>5): 
            try:
                datain[i,j]=float(subline[2:len(subline)])
            except IndexError:
                #index Error, Don´t Care
                k=k
            j+=1
        elif (subline[0:2] == "z:" and len(subline)<8 and len(subline)>5):
            try:
                datain[i,j]=float(subline[2:len(subline)])
            except IndexError:
                #index Error, Don´t Care
                k=k
            i+=1
            j=0
        else:
            k=k
    elif (i > (datanumber)):
        open('CSV/datain.csv', 'w').close()
        f= open("CSV/datain.csv","w+")
        f.write("x,y,z \r\n")
        for x1 in range(0, datanumber):
            f.write(str(datain[x1,0]) + ","+ str(datain[x1,1]) + "," + str(datain[x1,2])+"\r\n")
        f.close()
        fftransform(mpa,data_per_second,data_frecuency) 
        print("Frecuency data Saved " + str(k))
        if(k>=data_per_day):
            updateModel()
            print("Models Updated")
            k=0
        k+=1
        i=1
        j=0

        
def fftransform(mmaa,data_per_secondd,data_frecuencyy):

    datanumberr=data_per_secondd*60*data_frecuencyy

    datain  = np.array(pd.read_csv("CSV/datain.csv"), dtype='float64')
    datax= datain[:, 0]
    datay= datain[:, 1]
    dataz= datain[:, 2]
    
    N = datanumberr                #data obtained every 10 min for our device
    T = 1.0 / data_per_secondd     # sample spacing
    
    xfx = np.linspace(0.0, 1.0/(2.0*T), N//2)
    yfx = fft(datax)
    xfy = np.linspace(0.0, 1.0/(2.0*T), N//2)
    yfy = fft(datay)
    xfz = np.linspace(0.0, 1.0/(2.0*T), N//2)
    yfz = fft(dataz)
    frecx = (2.0/N * np.abs(yfx[0:N//2]))
    frecy = (2.0/N * np.abs(yfy[0:N//2]))
    frecz = (2.0/N * np.abs(yfz[0:N//2]))
    maxfrecx=xfx[frecx.argmax()]
    if ((frecx[frecx.argmax()].astype('float32')) == float("inf")):
        maxampx=np.float64(0.0001)
    else:
        maxampx=frecx[frecx.argmax()]
    
    maxfrecy=xfy[frecy.argmax()]
    if ((frecy[frecy.argmax()].astype('float32')) == float("inf")):
        maxampy=np.float64(0.0001)
    else:
        maxampy=frecy[frecy.argmax()]
        
    maxfrecz=xfz[frecz.argmax()]
    if ((frecz[frecz.argmax()].astype('float32')) == float("inf")):
        maxampz=np.float64(0.0001)
    else:
        maxampz=frecz[frecz.argmax()]
        
    rfx=maxampx/mmaa # This is the risk factor 
    rfy=maxampy/mmaa # This is the risk factor 
    rfz=maxampz/mmaa # This is the risk factor 
    fi=open("CSV/frec.csv", "a+")
    fi.write(str(maxfrecx.astype('float32'))+","+str(maxampx.astype('float32'))+ "," + str(maxfrecy.astype('float32'))+","+str(maxampy.astype('float32'))+ "," + str(maxfrecz.astype('float32'))+","+str(maxampz.astype('float32'))+ "," + str(rfx.astype('float32')) + "," + str(rfy.astype('float32')) + ","+ str(rfz.astype('float32'))+ "\r\n")
    fi.close()
    jsonDocument = {
        "MaxFx"  : maxfrecx,
        "MaxAx"  : maxampx,
        "MaxFy"  : maxfrecy,
        "MaxAy"  : maxampy,
        "MaxFz"  : maxfrecz,
        "MaxAz"  : maxampz,
        "RiskFx" : rfx,
        "RiskFy" : rfy,
        "RiskFz" : rfz,
    }
    clientmqtt.publishEvent("Risk", "json", {"d":{"Riskx": rfx}})
    clientmqtt.publishEvent("Risk", "json", {"d":{"Risky": rfy}})
    clientmqtt.publishEvent("Risk", "json", {"d":{"Riskz": rfz}})
    myDatabaseDemo.create_document(jsonDocument)
    
def updateModel():

    print("Training Model")
    data_train = pd.read_csv("CSV/frec.csv")
    global mpa
    API_KEY="YOUR_PUSHETA_API"
    p=Pushetta(API_KEY)

    xlnk = Xlnk()
    overlay = Overlay('/home/xilinx/pynq/overlays/sort/multi.bit')
    dma = overlay.mult_dma

    data_train = pd.read_csv("CSV/frec.csv")   

    open('CSV/frec.csv', 'w').close()
    fil= open("CSV/frec.csv","w+")
    fil.write("frecx,ampx,frecy,ampy,frecz,ampz,riskx,risky,riskz\r\n")
    fil.close()

    labelsx = data_train['riskx']
    labelsy = data_train['risky']
    labelsz = data_train['riskz']

    arrfrecx= data_train['frecx']
    arrfrecy= data_train['frecy']
    arrfrecz= data_train['frecz']
    arrampx = data_train['ampx']
    arrampy = data_train['ampy']
    arrampz = data_train['ampz']

    mfrecx= 0
    mfrecy= 0
    mfrecz= 0
    mampx = 0
    mampy = 0
    mampz = 0

    trainx = data_train.drop(['frecy','frecz','ampy','ampz','riskx','risky','riskz'],axis=1)
    trainy = data_train.drop(['frecx','frecz','ampx','ampz','riskx','risky','riskz'],axis=1)
    trainz = data_train.drop(['frecx','frecy','ampx','ampy','riskx','risky','riskz'],axis=1)

    x_trainx , dontcare1x , y_trainx , dontcare2x = train_test_split(trainx , labelsx , test_size = 1,random_state =2)
    x_trainy , dontcare1y , y_trainy , dontcare2y = train_test_split(trainy , labelsy , test_size = 1,random_state =2)
    x_trainz , dontcare1z , y_trainz , dontcare2z = train_test_split(trainz , labelsz , test_size = 1,random_state =2)

    filenamex = 'Models/bridgex.sav'
    filenamey = 'Models/bridgey.sav'
    filenamez = 'Models/bridgez.sav'    

    modelx = pickle.load(open(filenamex, 'rb'))
    modely = pickle.load(open(filenamey, 'rb'))
    modelz = pickle.load(open(filenamez, 'rb'))

    modelx.fit(x_trainx, y_trainx)
    print("Model 1 Fit")
    modely.fit(x_trainy, y_trainy)
    print("Model 2 Fit")
    modelz.fit(x_trainz, y_trainz)
    print("Model 3 Fit")

    in_buffer1 = xlnk.cma_array(shape=(len(labelsx),), dtype=np.float32)
    in_buffer2 = xlnk.cma_array(shape=(len(labelsx),), dtype=np.float32)
    in_buffer3 = xlnk.cma_array(shape=(len(labelsx),), dtype=np.float32)
    in_buffer4 = xlnk.cma_array(shape=(len(labelsx),), dtype=np.float32)
    in_buffer5 = xlnk.cma_array(shape=(len(labelsx),), dtype=np.float32)
    in_buffer6 = xlnk.cma_array(shape=(len(labelsx),), dtype=np.float32)
    out_buffer = xlnk.cma_array(shape=(len(labelsx),), dtype=np.float32)

    for i in range(1,len(labelsx)-1):
        in_buffer1[i]=arrfrecx[i]
        in_buffer2[i]=arrfrecy[i]
        in_buffer3[i]=arrfrecz[i]
        in_buffer4[i]=arrampx[i]
        in_buffer5[i]=arrampy[i]
        in_buffer6[i]=arrampz[i]

    dma.sendchannel.transfer(in_buffer1)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    mfrecx=out_buffer[len(out_buffer)-1]
    overlay = Overlay('/home/xilinx/pynq/overlays/sort/multi.bit')
    dma = overlay.mult_dma
    dma.sendchannel.transfer(in_buffer2)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    mfrecy=out_buffer[len(out_buffer)-1]
    overlay = Overlay('/home/xilinx/pynq/overlays/sort/multi.bit')
    dma = overlay.mult_dma
    dma.sendchannel.transfer(in_buffer3)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    mfrecz=out_buffer[len(out_buffer)-1]
    overlay = Overlay('/home/xilinx/pynq/overlays/sort/multi.bit')
    dma = overlay.mult_dma
    dma.sendchannel.transfer(in_buffer4)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    mampx=out_buffer[len(out_buffer)-1]
    overlay = Overlay('/home/xilinx/pynq/overlays/sort/multi.bit')
    dma = overlay.mult_dma
    dma.sendchannel.transfer(in_buffer5)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    mampy=out_buffer[len(out_buffer)-1]
    overlay = Overlay('/home/xilinx/pynq/overlays/sort/multi.bit')
    dma = overlay.mult_dma
    dma.sendchannel.transfer(in_buffer6)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    mampz=out_buffer[len(out_buffer)-1]

    datax = [[mfrecx,mampx]]
    dfx = pd.DataFrame(datax,columns=['frecx','ampx'])
    datay = [[mfrecy,mampy]]
    dfy = pd.DataFrame(datay,columns=['frecy','ampy'])
    dataz = [[mfrecz,mampz]]
    dfz = pd.DataFrame(dataz,columns=['frecz','ampz'])

    y_predx = modelx.predict(dfx)
    y_predy = modely.predict(dfy)
    y_predz = modelz.predict(dfz)
    
    if (y_predx[0] >= mpa):
        p.pushMessage("Bridge Baby Monitor", "Hello human, I do not feel well, could you give me maintenance? :( ")
        clientmqtt.publishEvent("Signal", "json", {"d":{"Value": "Critical!"}})
    elif (y_predy[0] >= mpa):
        p.pushMessage("Bridge Baby Monitor", "Hello human, I do not feel well, could you give me maintenance? :( ")
        clientmqtt.publishEvent("Signal", "json", {"d":{"Value": "Critical!"}})
    elif (y_predz[0] >= mpa):
        p.pushMessage("Bridge Baby Monitor", "Hello human, I do not feel well, could you give me maintenance? :( ")
        clientmqtt.publishEvent("Signal", "json", {"d":{"Value": "Critical!"}})
    else:
        p.pushMessage("Bridge Baby Monitor", "Hello human, today I feel great! :3")
        clientmqtt.publishEvent("Signal", "json", {"d":{"Value": "Excelent!"}})
        

    pickle.dump(modelx, open(filenamex, 'wb'))
    pickle.dump(modely, open(filenamey, 'wb'))
    pickle.dump(modelz, open(filenamez, 'wb'))

    
Connected = False   #global variable for the state of the connection

mensaje=""
 
broker_address= "localhost"  #Broker address
port = 1883                        #Broker port
 
client = mqttClient.Client("Python")               #create new instance
client.on_connect= on_connect                      #attach function to callback
client.on_message= on_message                      #attach function to callback

num = 1
 
client.connect(broker_address, port=port)          #connect to broker
 
client.loop_start()        #start the loop
 
while Connected != True:    #Wait for connection
    time.sleep(0.1)
 
client.subscribe("Data")

try:
    while True:
        time.sleep(1)
        
except KeyboardInterrupt:
    print ("exiting")
    client.disconnect()
    client.loop_stop()
    clientmqtt.disconnect()